<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Crack_The_Code_Sep_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://dmcommunity.org/wp-content/uploads/2019/08/crackthecode.png)

1. 682 – one number is correct and in the correct position
2. 645 – one number is correct but in the wrong position
3. 206 – two numbers are correct but in the wrong positions
4. 738 – nothing is correct
5. 780 – one number is correct but in the wrong position.

In [3]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from pyomo.contrib.latex_printer import latex_printer


In [63]:
model = ConcreteModel()

model.I = RangeSet(1,3)
model.J = RangeSet(0,9)

model.X = Var(model.I, model.J, within = Binary)

def con0_rule(model,i):
  return sum(model.X[i,j] for j in model.J) == 1
model.con0 = Constraint(model.I, rule = con0_rule)

#682 – one number is correct and in the correct position
model.con1 = Constraint(expr = model.X[1,6] + model.X[2,8] + model.X[3,2] == 1)
model.con11 = Constraint(expr = sum(model.X[i,6] +  model.X[i,8] + model.X[i,2] for i in model.I) == 1)

#645 one number is correct but in the wrong position
def con2_rule(model):
  return sum(model.X[i,j] for i in model.I for j in [6,4,5]) == 1
model.con2 = Constraint(rule = con2_rule)
model.con22 = Constraint(expr = sum(model.X[i,6] +  model.X[i,4] + model.X[i,5] for i in model.I) == 1)


#206 – two numbers are correct but in the wrong positions
def con3_rule(model):
  return sum(model.X[i,j] for i in model.I for j in [2,0,6]) == 2
model.con3 = Constraint(rule = con3_rule)

#738 – nothing is correct
model.con4 = Constraint(expr = sum(model.X[i,j] for i in model.I for j in [7,3,8]) == 0)

#780 – one number is correct but in the wrong position.
def con5_rule(model):
  return sum(model.X[i,j] for i in model.I for j in [7,8,0]) == 1
model.con5 = Constraint(rule = con5_rule)
model.con55 = Constraint(expr = sum(model.X[i,7] +  model.X[i,8] + model.X[i,0] for i in model.I) == 1)

In [ ]:
model.pprint()

In [64]:
# Solve model
opt = SolverFactory('cbc')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

Solution is Optimal
Solve Time:  0.0


In [66]:
x = model.X.extract_values()
sol = {}
for i,j in x.keys():
  if x[i,j] > 0:
    sol[i] = j

print("Final Solution: {}".format(list(sol.values())))

Final Solution: [0, 5, 2]
